<a href="https://www.kaggle.com/code/ukaszniedwiadek/food101-exodia?scriptVersionId=178383027" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/ukaszniedwiadek/food101-exodia?scriptVersionId=177471599" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



import torch
import os
import csv
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def load_img(path):
    img_bgr = cv2.imread(path)
    img_rgb = img_bgr[:, :, ::-1]
    return img_rgb


class CustomDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, give_label=True):
        """Performed only once when the Dataset object is instantiated.
        give_label should be False for test data
        """ 
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.data_root = data_root
        self.transforms = transforms
        self.give_label = give_label
        
        if give_label == True:
            self.df['label'] = self.df['label'].astype(int)
            self.labels = self.df['label'].values

    def __len__(self):
        """Function to return the number of records in the dataset
        """ 
        return self.df.shape[0]
    
    def __getitem__(self, index):
        """Function to return samples corresponding to a given index from a dataset
        """ 
        # get labels
        if self.give_label:
            target = self.labels[index]
            target = torch.tensor(target)

        # Load images
        img  = load_img(f'{self.data_root}/{self.df.loc[index]["image_id"]}.jpg').astype(np.float32)
        # img /= 255.0 # Normalization

        # Transform images
        if self.transforms:
            img = self.transforms(image=img)['image']

        if self.give_label == True:
            return img, target
        else:
            return img

def get_labels(path, give_label):
    list_id = []
    list_label = []
    with open(path, mode ='r')as file:
        csvFile = csv.reader(file)
        for lines in csvFile:
            list_id.append(lines[0])
            if give_label:
                list_label.append(lines[1])
    list_id.pop(0)
    if give_label:
        list_label.pop(0)
        return list_id, list_label
    return list_id

main_dir = "/kaggle/input/dat18seefood"
batch_size = 32
image_size = 256




items = os.listdir(main_dir)
train_id = []
train_label = []

test_id = []
test_label = []

for item in items:
    if item == "train.csv":
        path = os.path.join(main_dir, item)
        train_id, train_label = get_labels(path, give_label=True)
    if item == "test.csv":
        test_id = get_labels(path, give_label=False)
        


X_train, X_val, y_train, y_val = train_test_split(train_id, train_label, stratify=train_label, test_size=0.20, random_state=42)
print(len(X_val))
print(len(X_train))
df_train = pd.DataFrame({
    'image_id': X_train,
    'label': y_train
})

df_val = pd.DataFrame({
    'image_id': X_val,
    'label': y_val
})

df_test = pd.DataFrame({
    'image_id': test_id,
})

df = pd.read_csv(main_dir + '/labelnames.csv')
print(df)

image_transforms = {
    'train':
    A.Compose(([
        A.Resize(image_size,image_size,p=1),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.CoarseDropout(p=0.5),
        ToTensorV2(p=1.0)
    ])),
    'valid':
    A.Compose(([
        A.Resize(image_size,image_size,p=1),
        ToTensorV2(p=1.0)
    ])),
    'test':
    A.Compose(([
        A.Resize(512,512,p=1),
        ToTensorV2(p=1.0)
    ]))
}

train_dataset = CustomDataset(df_train, main_dir+"/train/",transforms=image_transforms["train"],give_label=True)
valid_dataset = CustomDataset(df_val, main_dir+"/train/",transforms=image_transforms["valid"],give_label=True)
test_dataset = CustomDataset(df_test, main_dir+"/test/",transforms=image_transforms["test"],give_label=False)

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': DataLoader(valid_dataset, batch_size=batch_size, shuffle=True),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
}
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
print(features.shape, labels.shape)


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

15150
60600
     label       labelname
0        0       Apple pie
1        1  Baby back ribs
2        2         Baklava
3        3  Beef carpaccio
4        4    Beef tartare
..     ...             ...
96      96           Tacos
97      97        Takoyaki
98      98        Tiramisu
99      99    Tuna tartare
100    100         Waffles

[101 rows x 2 columns]
torch.Size([32, 3, 256, 256]) torch.Size([32])


In [ ]:
print("JD Disa zwisa")

In [1]:
import torch

print(torch.cuda.is_available())

True
